# Forward Rendering

This notebook shows how solve the radiative transfer using SHDOM and render a simulated image.

The source Fortran code was created by Frank Evans and it is part of the SHDOM package: http://nit.colorado.edu/shdom.html

The python wrapper was created by Amit Aides and Aviad Levis, Technion inst. of Technology 2019: https://github.com/aviadlevis/pyshdom

In [1]:
""" Description """

import shdom
import numpy as np

scene_params = shdom.SceneParameters()
numerical_params = shdom.NumericalParameters()

rte_solver = shdom.RteSolver(scene_params=scene_params,
                             numerical_params=numerical_params)

In [3]:
from shdom import Mie

wavelength_band = (0.672, 0.672)
particle_type='Water' 
distribution='Gamma'
alpha=7.0
num_effective_radii=100
start_effective_radius=1.0
end_effective_radius=25.0
max_integration_radius=75.0


mie = Mie(wavelength_band=wavelength_band,
          particle_type=particle_type,
          distribution=distribution,
          alpha=alpha)

mie.compute_table(
    num_effective_radii=num_effective_radii,
    start_effective_radius=start_effective_radius,
    end_effective_radius=end_effective_radius,
    max_integration_radius=max_integration_radius)

In [4]:
np.ones(shape=[3]).shape


(3,)